In [1]:
import numpy as np # linear algebra
import pandas as pd # data manipulation
import sys

### Hier werden n Datensets eingelesen, muss vorerst manuell angepasst werden

In [3]:
#1 bis n Datensets einlesen
dax1 = pd.read_csv('dax_data/dax.csv')
dax2 = pd.read_csv('dax_data/GDAXI.csv')
#check
dax1.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,28/05/2013,8432.459961,8526.419922,8424.400391,8480.870117,8480.870117,97455000.0
1,29/05/2013,8437.209961,8445.459961,8316.700195,8336.580078,8336.580078,107502500.0
2,30/05/2013,8314.200195,8416.360352,8293.959961,8400.200195,8400.200195,103141400.0
3,31/05/2013,8385.379883,8401.629883,8297.030273,8348.839844,8348.839844,122400300.0
4,03/06/2013,8291.490234,8395.559570,8214.780273,8285.799805,8285.799805,97748300.0


In [4]:
countDataSets = 2

# Definition der Methoden

### Quality

In [5]:
#NullValues zählen
def howManyNullValues( oneDataSet ):
    return oneDataSet.isnull().sum().sum()

In [6]:
#Werteanzahl bestimmen
def howManyValues( oneDataSet ):
    return oneDataSet.shape[0] * oneDataSet.shape[1]

In [7]:
#Anteil von Nullvalues an Values bestimmen
def calculateRatioNullValues( oneDataSet ):
    return 100 - ((howManyNullValues( oneDataSet ) / howManyValues( oneDataSet )) * 100)

In [8]:
#Größe des Datensets bestimmen (Methode 1)
def getSizeMethodOne( oneDataSet ):
    return oneDataSet.memory_usage(index=True).sum()/1000

In [9]:
#Größe des Datensets bestimmen (Methode 2)
def getSizeMethodTwo( oneDataSet ):
    return sys.getsizeof(oneDataSet)/1000

In [10]:
# bestimme für ein Datenset alle Qualitätsparameter
def getQualityParameters( dataset, dsindex):
    
    data = [(dsindex, howManyValues(dataset),howManyNullValues(dataset),calculateRatioNullValues(dataset),getSizeMethodOne(dataset),getSizeMethodTwo(dataset))]
    qualityParameterForDataSet = pd.DataFrame(data,columns=['dataset_id','sum_Values', 'sum_NullValues', 'ratio_NullValues', 'filesize_One', 'filesize_Two'])

    return qualityParameterForDataSet

In [11]:
def getQualityRanks(qualitydataframe):
    
    dataframe = qualitydataframe #the original one should be untouched
    
    dataframe['sum_Values'] = dataframe['sum_Values'].rank(ascending=False)
    dataframe['sum_NullValues'] = dataframe['sum_NullValues'].rank(ascending=True)
    dataframe['ratio_NullValues'] = dataframe['ratio_NullValues'].rank(ascending=False)
    dataframe['filesize_One'] = dataframe['filesize_One'].rank(ascending=False)
    dataframe['filesize_Two'] = dataframe['filesize_Two'].rank(ascending=False)
    
    return dataframe

### Kosten

In [12]:
#finde im von max nach min sortierten Kostenparameter-Datenset die Position des gegebenen Datensets (Rang)
def getCostRank(index):
    return costParameters.loc[costParameters['id'] == index].index.values.astype(int)[0]

### Aufwand

In [13]:
#Aufwand nach gesetzten Regeln bestimmen
def getScoreDataPreparation( oneDataSet ):
    scoreDataPreparation = 10

    scoreEntries = calculateRatioNullValues(oneDataSet)
    
    if scoreEntries != 100:
        scoreDataPreparation = scoreDataPreparation - 1
    if scoreEntries < 85:
        scoreDataPreparation = scoreDataPreparation - 1
    if scoreEntries < 70:
        scoreDataPreparation = scoreDataPreparation - 1
    
    #hier fehlt Zeugs, welches für den Anwendungsfall aber noch nicht relevant war
    
    # genauer: DateType - Bestimmung
    # StringType-Bestimmung
    
    return scoreDataPreparation 

In [14]:
#bestimme die Werte der vier Kriterien
def getScoreDataFrameForDataSet ( oneDataSet, datasetindex, importance, costRank, qualityRank ):
    
    dataScore = [(datasetindex, importance, costRank, qualityRank, getScoreDataPreparation(oneDataSet))]
    scoresDataSet = pd.DataFrame(dataScore,columns=['dataset_id','importance_score','cost_score', 'quality_score', 'hassle_score'])
    
    return scoresDataSet

# Programm

In [15]:
#Qualitätsparameter für jedes Datenset erhalten
qualityParameters = pd.DataFrame()

qualityParameters = getQualityParameters(dax1, "kaggle-dax")
qualityParameters = qualityParameters.append(getQualityParameters(dax2,"yahoo-dax"))

qualityParameters = qualityParameters.reset_index(drop = True)

qualityParameters.head()

,dataset_id,sum_Values,sum_NullValues,ratio_NullValues,filesize_One,filesize_Two
0,kaggle-dax,9128,246,97.304996,73.104,150.064
1,yahoo-dax,168,11,93.452381,1.424,2.864


In [16]:
#nutze QUalitätsparameter zur Bestimmung eines Gesamtrangs

qualityParameters = getQualityRanks(qualityParameters)

#qualityParameters["sumranks"] = qualityParameters.sum(axis=1)
qualityParameters["sumranks"] = qualityParameters.apply(lambda row: row['sum_Values' : 'filesize_Two'].sum(),axis=1)
qualityParameters['quality_score'] = qualityParameters['sumranks'].rank(ascending=False)

qualityParameters.head()

,dataset_id,sum_Values,sum_NullValues,ratio_NullValues,filesize_One,filesize_Two,sumranks,quality_score
0,kaggle-dax,1.0,2.0,1.0,1.0,1.0,6.0,2.0
1,yahoo-dax,2.0,1.0,2.0,2.0,2.0,9.0,1.0


In [17]:
costdata = [("kaggle-dax",0),("yahoo-dax",0)]
costParameters = pd.DataFrame(costdata,columns=['dataset_id','costs'])

costParameters['cost_score'] = costParameters['costs'].rank(ascending=False)

costParameters.head()

,dataset_id,costs,cost_score
0,kaggle-dax,0,1.5
1,yahoo-dax,0,1.5


In [18]:
scoreDataSet = getScoreDataFrameForDataSet(dax1, "kaggle-dax", 5, (costParameters.loc[costParameters['dataset_id'] == "kaggle-dax", 'cost_score']).item(), (qualityParameters.loc[qualityParameters['dataset_id'] == "kaggle-dax", 'quality_score']).item())
scoreDataSet = scoreDataSet.append(getScoreDataFrameForDataSet(dax2, "yahoo-dax", 5, (costParameters.loc[costParameters['dataset_id'] == "yahoo-dax", 'cost_score']).item(), (qualityParameters.loc[qualityParameters['dataset_id'] == "yahoo-dax", 'quality_score']).item()))
scoreDataSet.head()

,dataset_id,importance_score,cost_score,quality_score,hassle_score
0,kaggle-dax,5,1.5,2.0,9
0,yahoo-dax,5,1.5,1.0,9


In [19]:
scoreDataSet["importance_score"] = (scoreDataSet["importance_score"]*(countDataSets/5))
scoreDataSet["hassle_score"] = (scoreDataSet["hassle_score"]*(countDataSets/10))

scoreDataSet["score"] = scoreDataSet["importance_score"] + scoreDataSet["hassle_score"] + scoreDataSet["quality_score"] + scoreDataSet["cost_score"]
scoreDataSet["score_rank"] = scoreDataSet['score'].rank(ascending=False)

scoreDataSet.head()

,dataset_id,importance_score,cost_score,quality_score,hassle_score,score,score_rank
0,kaggle-dax,2.0,1.5,2.0,1.8,7.3,1.0
0,yahoo-dax,2.0,1.5,1.0,1.8,6.3,2.0


In [20]:
print("Das empfohlene Datenset ist das Datenset mit der ID: ", (scoreDataSet.loc[scoreDataSet['score_rank'] == 1, 'dataset_id']).item())

Das empfohlene Datenset ist das Datenset mit der ID:  kaggle-dax
